1. Сравните метрики hit_rate@k, precision@k.   
Какую метрику использовать предпочтительно и почему?  
Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?


###  precision@k - предпочтительнее так как указывается доля предсказанных товаров, есть информативность о модели и эффективность системы определяется(если необходима такая эффективность).

Hit_rate@k - уместно использовать в задачах если требуеться класифицировать присутствия/отсутствия взоимодействий среди множест пользователь/товар[:k].
(подбор похожих товаров, товар-user, товар-item),
(подбор похожих пользователей, пользователь-user, пользователь-item).

Не могу придумать задачь, которые бы давали информативность о модели при метрике hit_rate@k...

2. В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в  логарифм.



Логарифм в числителе что бы вес давал меньшую разнику в случае удаения он начала рекомендативного списка(разница между 100 и 101 элементами рекомендативного списка будет меньше чем разница между 1 и 10 элементами)

3. Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики **и чем являются интеракции**, чтобы она могла быть использована).

Посмотрел в интернете:

Ранговый коэффициент корреляции Кендэлла,

Ранговый коэффициент корреляции Спирмена,

Expected reciprocal rank,

PFound

4. boughted = [1, 3, 5, 7, 9, 11]  
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  
Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, MRR@8,  
Сначала поссчитайет "руками", а затем подтвердите рассчеты с помощью функций (некоторые были на семинаре, некоторые нужно будут написать самостоятельно)

IDCG должен считаться для всех 8 позиций рекомендаций

**Пример выполнения 4го задания**

MSE на  
$y = [0, 2]$  
$y_{pred} = [1, 2]$

MSE = $0.5*((0 - 1)^2 + (2-2)^2) = 0.5$ 

In [1]:
import numpy as np
((np.array([0, 2]) - np.array([1, 2]))**2).mean()

0.5

#### Функции

In [2]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    
    return precision


def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k] # добавлен срез до k
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(0, k):    # изменен range(1, k+1) -> range(0, k)
        if flags[i] == True:    # вышестоящее изменения для проверки по индексу(первый элемент==0)
            p_k = precision_at_k(
                recommended_list,
                bought_list,
                k=i+1
            ) # k=i -> k=i+1(для коректной работы функции pr@k)
            sum_ += p_k
            
    result = sum_ / len(flags)
    
    return result


def ndcg_at_k(recommended_list, bought_list, k=False):

    if k:
        recommended_list=recommended_list[:k]
    recommended_list= np.array(recommended_list)
    bought_list = np.array(bought_list)
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    _ideal = []
    for i in range(0, k):  
        if i == 0 or i == 1:
            _ideal.append(1)
        else:
            _ideal.append(1/np.log2(i+1))
            
    sum_flags = np.dot(_ideal, flags)
    result = sum_flags / np.sum(_ideal)
    
    return result


def reciprocal_rank(recommended_list, bought_list, k=False):
    ranks=0.
    if k:
        recommended_list=recommended_list[:k]
    for i, item_rec in enumerate(recommended_list):
        for item_bought in bought_list:
            if item_rec == item_bought:
                ranks += 1 / (i+1)
    return ranks / len(recommended_list)




In [3]:
boughted = [1, 3, 5, 7, 9, 11]
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]

boughted = np.array(boughted)
recommended = np.array(recommended)
recomm_8 = recommended[:8]

In [4]:
recomm_8, boughted

(array([ 2,  5,  7,  4, 11,  9,  8, 10]), array([ 1,  3,  5,  7,  9, 11]))

### Precision@8

In [6]:
pr_8_flags = np.isin(boughted, recomm_8)
pr_8_sum = np.sum(pr_8_flags)

In [7]:
pr_8_flags, pr_8_sum

(array([False, False,  True,  True,  True,  True]), 4)

In [8]:
len(recomm_8)

8

In [9]:
pr_8 = pr_8_sum/len(recomm_8)
pr_8

0.5

In [10]:
precision_at_k(recommended_list=recommended, bought_list=boughted,k=8)

0.5

### Recall@8

In [11]:
rec_8_flags = np.isin(boughted, recomm_8)
rec_8_sum = np.sum(rec_8_flags)

In [12]:
rec_8 = rec_8_sum/ len(boughted)

In [13]:
rec_8


0.6666666666666666

In [14]:
recall_at_k(recommended_list=recommended, bought_list=boughted,k=8)

0.6666666666666666

### AP@k - average precision at k

In [16]:
ap_flags = np.isin(recommended[:8], boughted)
ap_flags

array([False,  True,  True, False,  True,  True, False, False])

#### pr_1...pr_8

In [18]:
pr_1 = (0)*0
pr_2 = 1/2
pr_3 = 2/3
pr_4 = (2/4)*0
pr_5 = 3/5
pr_6 = 4/6
pr_7 = (4/7)*0
pr_8 = (4/8)*0
len_rec = len(recomm_8)

pr_1_8 = sum([0,pr_2,pr_3,0,pr_5,pr_6,0,0])/len_rec
pr_1_8

0.30416666666666664

In [19]:
ap_k(recommended_list=recommended, bought_list=boughted, k=8)

0.30416666666666664

### NDCG@k Normalized discounted cumulative gain

In [20]:
recomm_8, boughted

(array([ 2,  5,  7,  4, 11,  9,  8, 10]), array([ 1,  3,  5,  7,  9, 11]))

In [21]:
dcg_8=(1/8)*(0/1+1/np.𝑙𝑜𝑔2(2)+1/np.𝑙𝑜𝑔2(3)+0/np.𝑙𝑜𝑔2(4)+1/np.𝑙𝑜𝑔2(5)+1/np.𝑙𝑜𝑔2(6)+0/np.𝑙𝑜𝑔2(7)+0/np.𝑙𝑜𝑔2(8))

𝑖𝑑𝑒𝑎𝑙dcg_8=(1/8)*(1/1+1/np.𝑙𝑜𝑔2(2)+1/np.𝑙𝑜𝑔2(3)+1/np.𝑙𝑜𝑔2(4)+1/np.𝑙𝑜𝑔2(5)+1/np.𝑙𝑜𝑔2(6)+1/np.𝑙𝑜𝑔2(7)+1/np.𝑙𝑜𝑔2(8))

ndcg_8=dcg_8/𝑖𝑑𝑒𝑎𝑙dcg_8

ndcg_8

0.5279127445637272

In [22]:
ndcg_at_k(recommended, boughted, k=8)

0.527912744563727

### MRR@k Mean Reciprocal Rank at k

In [23]:
recomm_8, boughted

(array([ 2,  5,  7,  4, 11,  9,  8, 10]), array([ 1,  3,  5,  7,  9, 11]))

In [24]:
rr_1 = 0
rr_2 = 1/2
rr_3 = 1/3
rr_4 = 0
rr_5 = 1/5
rr_6 = 1/6
rr_7 = 0
rr_8 = 0

In [25]:
rr_list = [rr_1,rr_2,rr_3,rr_4,rr_5,rr_6,rr_7,rr_8]

In [26]:
mrr_at_8 = sum(rr_list)/len(rr_list)
mrr_at_8

0.15

In [27]:
reciprocal_rank(recommended_list=recommended, bought_list=boughted, k=8)

0.15